# 演習の方針

1. **ベースラインモデル評価**  
   素のモデルで回答を生成し、講義内容との整合性の低さを観察します。これにより、特別な学習なしでのモデルの限界を確認します。

2. **文字起こしデータの活用**  
   講義の文字起こしデータを導入し、モデルが講義内容を参照した回答を生成する傾向を観察します。ただし、Retrieval（情報検索）精度の限界から結果は不安定になる可能性があります。

3. **チャンク化の導入**  
   文字起こしデータをチャンク（小単位）に分割し、より安定して関連コンテンツを取得できるようにします。この段階では文脈理解にまだ課題があることを確認します。

4. **Rerankの適用**  
   検索結果のランク付けを導入し、より的確で安定した回答を目指します。

5. **応用改善手法**  
   文字起こしの品質向上のための編集技術や、メタデータの活用による性能向上手法を探ります。

## 扱う質問

「Inference Time Scaling（推論時スケーリング）」に関する質問を取り扱います。これは以下の背景を持つトピックです。

- 2024年8月発表の論文「Scaling LLM Test-Time Compute Optimally can be More Effective than Scaling Model Parameters」で提唱された概念
- OpenAIのGPT-o1（2024年9月リリース）で実用化され、注目を集めた比較的新しいアプローチ
- 2024年度LLM講座の第4回講義でも取り上げられた重要テーマ

## 扱うモデル

Meta-Llama-3-8B-Instruct（2024年4月リリース）を使用します。このモデルは、リリース時期の関係上、以下の特徴を持ちます。

- 「Inference Time Scaling」の概念が広まる前に訓練されており、このトピックに関する知識を持たないと想定される
- この特性を活かし、純粋なベースライン評価から各手法の効果を観察する

### 演習環境の準備

In [ ]:
!pip install --upgrade transformers
!pip install google-colab-selenium
!pip install bitsandbytes

In [ ]:
# 演習用のコンテンツを取得
!git clone https://github.com/matsuolab/lecture-ai-engineering.git

In [ ]:
# HuggingFace Login
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
# CUDAが利用可能ならGPUを、それ以外ならCPUをデバイスとして設定
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import random
random.seed(0)

In [ ]:
# モデル(Llama3)の読み込み

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map="auto",
            quantization_config=bnb_config,
            torch_dtype=torch.bfloat16,
        )

# 1. ベースラインモデル評価
**まずはベースモデルがどの程度知識を持っているか確かめる**

In [ ]:
messages = [
    {"role": "system", "content": "質問に回答してください。必ず「日本語で回答」すること。"},
    {"role": "user", "content": "LLMにおけるInference Time Scalingとは？"},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    # max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=False,
    # temperature=0.6, # If do_sample=True
    # top_p=0.9,  # If do_sample=True
)

In [ ]:
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

## 結果 (ベースモデル)

Meta-Llama-3-8B-Instructは「Inference Time Scaling」について誤った知識を提示しました：
* モデルは従来の「推論時間の短縮」という文脈でInference Time Scalingを解釈しており、これはLLM分野における最新の「Inference Time Scaling」概念（推論時計算資源の最適配分）とは異なる説明になります。

---

# 2. 文字起こしデータの活用
## 講義内容をソースとして活用 (RAG導入)

モデルの回答の事実性を向上させるためにRetrieval Augmented Generation (RAG)技術を導入します：

* **知識ソース**: LLM講座第4講における講師の発言内容
* **目的**: モデルに「Inference Time Scaling」に関する正確な知識と文脈を提供し、事実に基づいた回答を促す

**初期RAG実装（ベーシックアプローチ）**:
* **ドキュメント処理**: 音声認識モデル(speech2text)で書き起こした生テキストをそのまま使用
* **分割方法**: 「。」（句点）で区切られた文単位でテキストを分割
* **検索手法**: シンプルな類似度ベースの検索でクエリに関連する文を抽出
* **制約条件**: モデルの入力トークン制限に収まるよう関連文のみを選択

In [ ]:
from sentence_transformers import SentenceTransformer

emb_model = SentenceTransformer("infly/inf-retriever-v1-1.5b", trust_remote_code=True)
# In case you want to reduce the maximum length:
emb_model.max_seq_length = 8192

In [55]:
with open("/content/lecture-ai-engineering/day3/data/LLM2024_day4_raw.txt", "r") as f:
  raw_writedown = f.read()

In [56]:
# ドキュメントを用意する。
documents = [text.strip() for text in raw_writedown.split("。")]
print("ドキュメントサイズ: ", len(documents))
print("ドキュメントの例: \n", documents[250])

ドキュメントサイズ:  306
ドキュメントの例: 
 このDecodingにもいろんな方法があってグリーンDecodingだと単純に一番いいやつを選んでいく、一番確率が高いやつ選んでいくので、すごい単純ですけど、こういうトップKeyを取るとかトップ系を取るとかして、最後に一番いいやつを選ぶみたいなことをすると、これも結局計算をたくさんしてることになるわけですね


In [57]:
# Retrievalの実行
question = "LLMにおけるInference Time Scalingとは？"

query_embeddings = emb_model.encode([question], prompt_name="query")
document_embeddings = emb_model.encode(documents)

# 各ドキュメントの類似度スコア
scores = (query_embeddings @ document_embeddings.T) * 100
print(scores.tolist())

[[61.5933952331543, 65.96569061279297, 59.183807373046875, 53.39189910888672, 51.560462951660156, 58.39970779418945, 60.753231048583984, 54.146785736083984, 61.974422454833984, 62.86030960083008, 58.38917541503906, 59.106529235839844, 55.34397506713867, 57.358680725097656, 59.760498046875, 58.14842987060547, 62.52417755126953, 57.50199508666992, 56.670326232910156, 59.72063064575195, 59.44031524658203, 64.46634674072266, 62.21947479248047, 60.338104248046875, 53.73780822753906, 64.26367950439453, 57.02449035644531, 62.45271682739258, 56.58358383178711, 56.53034973144531, 56.84971237182617, 54.066192626953125, 51.05952072143555, 51.928340911865234, 56.22751235961914, 56.70086669921875, 57.826595306396484, 57.070465087890625, 59.671504974365234, 59.159122467041016, 57.539249420166016, 52.95311737060547, 55.697792053222656, 54.768226623535156, 58.13568115234375, 61.39663314819336, 58.75926208496094, 61.92470169067383, 56.53297805786133, 58.76426696777344, 59.054481506347656, 53.0557403564

In [58]:
topk = 5
for i, index in enumerate(scores.argsort()[0][::-1][:topk]):
  print(f"取得したドキュメント{i+1}: (Score: {scores[0][index]})")
  print(documents[index], "\n\n")

取得したドキュメント1: (Score: 67.52583312988281)
Trasnformerの場合はスケール則がごめんなパラメータ数が横軸になってますけどこういうふうになると、LLMの場合の一掃にソヨンそうなのでそれぞれ計測と書くとこんなふうになりますよということで、Trasnformer以外のスケール則っていうのもあの研修をされて、深さについても検証してまして、これも他のモデルが何だったかちょっと忘れちゃったけどリスキーだったような気がしますけどそう変えたときにどういうふうな変化するかっていうのをこういった形でプロットするようなGENIACすることができます 


取得したドキュメント2: (Score: 66.03395080566406)
1月に論文としてまして、スケーリングLLMthisTimeコンピュート口真理ちゃん日は増えてるっていうことで、良いらしいというふうに言われてます 


取得したドキュメント3: (Score: 65.96569061279297)
あのスケールするっていうところではタイトルの通りなんですけど、ちょっとこれスケーリングPretrainingってなってるんですけれども、ちょっと最近はですね、このPretrainingだけではなくて、推論をスケールさせるというような話も出てきてましてせっかくなのでその最近の話題ということですレンジのスケーリングことでちょっとタイトル詐欺が入ってるんですけどPretrainingだけじゃない、スケーリングを扱うということでちょっと若干あのタイトル詐欺なんですけども、あの最近の話題ということで水土日のスケジュールについても話していきたいなと思っています 


取得したドキュメント4: (Score: 65.190673828125)
気にしながらっていうのの実例を出した方がわかりやすいと思うので、実際にこれ開発者じゃないので、あの結果を見て推論してるだけなんで嘘ついてるかもしれないですけど例えばLlama3の論文を持ってくると8Billon70Billon405Billonって層の数モデルDimension埋め込みの数次元ですね、フィードフォワードの次元、アテンションの数っていうのを、こういうふうにしたよっていうふうに言われてます 


取得したドキュメント5: (Score: 65.134

In [59]:
 #回答に役立つ該当の発言はreference[1871]〜に含まれてます。
references = "\n".join(["* " + documents[i] for i in scores.argsort()[0][::-1][:topk]])
messages = [
    {"role": "system", "content": "質問に回答してください。必ず「日本語で回答」すること。また、与えられる資料を参考にして回答すること。"},
    {"role": "user", "content": f"[参考資料]\n{references}\n\n[質問] LLMにおけるInference Time Scalingとは？"},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    # max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=False,
    # temperature=0.6, # If do_sample=True
    # top_p=0.9,  # If do_sample=True
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


In [60]:
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

LLM（Large Language Model）におけるInference Time Scalingは、LLMの推論時間をスケーリングすることで、モデルがより高速に推論できるようにすることを意味します。すなわち、LLMの推論時間を短縮することで、より多くの入力データを処理できるようにすることを目指しています。

このスケーリングは、Pretrainingの段階で行われることが多く、LLMのパラメータ数やモデルサイズを減らすことで実現されます。例えば、Llama3の論文では、8億のパラメータ数を減らすことで、推論時間を短縮することができたと報告されています。

Inference Time Scalingは、LLMの推論速度を向上させることで、実際のアプリケーションでの使用を可能にします。例えば、チャットボットや自然言語処理系のシステムなど、LLMを使用するアプリケーションでは、推論時間の短縮が大きな利点になります。


## 結果 (初期RAG実装)

講義内容のドキュメントを追加したにもかかわらず、モデルの回答には依然として以下の問題が見られます：
* 「高速に推論する」など、従来の一般的な推論最適化と「Inference Time Scaling」を混同した誤った解釈が継続
* 講義内容を参照しているものの、概念の本質を正確に捉えられていない

### 問題分析
以下の要因が考えられます：
1. **ドキュメント品質の問題**: 音声認識による文字起こしの精度不足
2. **検索精度の課題**: 単純な文単位の分割では文脈が失われ、関連性の高いドキュメント片を適切に取得できていない可能性

### 書き起こしテキストの品質改善

日本語の音声認識（speech2text）モデルは精度に課題があることが知られています。以下に「LLMにおけるInference Time Scalingとは？」に関連する講義内容の書き起こしテキストを比較します：

### 講義中の該当発言 (LLM講座Day4後半から抜粋)


<修正前>
---

講義に戻ります。ちょっと練習の時間もあるのであと20分ぐらいで駆け足になりますけど、最後最近のスケールトレンドって話で**生のGENIACLM**の話をして終わろうと思いですねちょっとモチベーションから話すと、ちょっと頭で考えてみてほしいとか見れば一瞬で思うとんですけどバナナの色は何ですかって言われたときと、今日の講義聞いた上で、**ゲームソフトの問題は何だと思いますか**って聞かれたとき、多分あの考えることが違うと思うんですね。**羽の色なんですか**っていうと一瞬黄色ですねもしかしたら緑かもしれないけどぐらいですかね物によるかなみたいなおもちゃだったら違うかもみたいな、だんだんあの、考えていくといろいろ出てくるかもしれないすけど、少なくとも**スケール足の問題なんだと思いますか**って聞かれたときに、今日の話からするとスケール則っていうのはこういうものだからどうだろうこの辺が問題かなみたいな考えとやっぱ思考としては違うってことは何となく思うかなと思います。なんか人間的にはこの二つって全然違うしあの、答えるのに必要な考え方っていうのも違うように思えるわけです。**スケールって言ってる7Gのスケール**って言ってるのはこういった形で、あの簡単なものについては簡単に答えてもいいですし、そうじゃなくて、あの考えなきゃいけない問題に対しては、考える時間を、に計算式を使うというふうにしたときに、これいいことがあるのかっていうような話になってます。二つで、ちょっと順番が前後しますけどこれの仕組みは言語モデルでも効果的ですかっていう話と、これをどう実現できるかっていう、こういう二つの話が最近のトレンドとして出てきています。効果的ですかっていうのが、最近**大湾**と呼ばれる論文が論文じゃないか、モデルが**オペル**から出ましたプレビューとして出てますけどこの法案で注目されていますこれあの**論文にROMってかブログ**にあるとイエスって右側が訓練時の計算資源をスケールさせたときに、初めて何かロジックのベンチマークがあるんですけどこれをがどうなったかで何となくスケールしてると右側がテストTimeコンピュートっていうふうに書いてると思うんすけど、**水温時**に計算資源を増やしたときあるモデルを使うんだけど、簡単に答える方法と深く考えて答える方法みたいでだんだんコース計算式を増やしていったときに、性能がどう変わるかっていうのでこれもスケールしていってるということがわかると思います。こういった形で、要は考える時間をどうやら推論時に使うと計算資源を推論使うのはいいことがありそうだということがわかります。


<修正後>
---


講義に戻ります。ちょっと演習の時間もあるのであと20分ぐらいで駆け足になりますけど、最後最近のスケールトレンドってことで**「推論時のスケーリング」**についての話をして終わろうと思います。モチベーションから話すと、ちょっと頭で考えてみてもらえれば一瞬でわかると思うとんですけど、「バナナの色は何ですかって言われたとき」と、今日の講義聞いた上で、**「スケール則の問題は何だと思いますか」**って聞かれたとき、多分あの考えることが違うと思うんですね。
**「バナナの色なんですか」**っていうと黄色ですね。もしかしたら緑かもしれないけど、物によるかなみたいな、おもちゃだったら違うかもみたいな、だんだんあの、考えていくといろいろ出てくるかもしれないすけど、少なくとも**「スケール則の問題なんだと思いますか」**って聞かれたときに、今日の話からするとスケール則っていうのはこういうものだから「どうだろう」「この辺が問題かな」みたいな考えとはやっぱ思考としては違うってことは何となく思うかなと思います。
なんか人間的にはこの二つって全然違うしあの、答えるのに必要な考え方っていうのも違うように思えるわけです。**推論時のスケールって言ってるのは**こういった形で、あの簡単なものについては簡単に答えてもいいですし、そうじゃなくて、深く考えなきゃいけない問題に対しては、考える時間に計算資源を使うというふうにしたときに、これいいことがあるのかっていうような話になってます。
これの仕組みは言語モデルでも効果的ですかっていう話と、これをどう実現できるかっていう、こういう二つの話が最近のトレンドとして出てきています。効果的ですかっていうのが、最近**o1**と呼ばれるモデルが**OpenAI**から出ました。プレビューとして出てますけどこのo1で注目されています。これあのo1の**論文ってかブログ**にある図で、左側が訓練時の計算資源をスケールさせたときに、AIMEというロジックのベンチマークがあるんですけど、accuracyがどうなったかというと、何となくスケールしてる。右側がtest-time computeっていうふうに書いてると思うんすけど、**推論時**に計算資源を増やしたときあるモデルを使うんだけど、簡単に答える方法と深く考えて答える方法みたいでだんだん計算資源を増やしていったときに、性能がどう変わるかっていうので、これもスケールしていってるということがわかると思います。
こういった形で、要は考える時間をどうやら推論時に使うと、つまり計算資源を推論時に使うのはいいことがありそうだということがわかります。






---
### 文字起こしの誤り

上記の比較からわかるように、音声認識による書き起こしには重大な誤りが多数含まれています：
* 「スケール則の問題」→「ゲームソフトの問題」
* 「o1」→「大湾」
といった明らかに文脈に合わない単語変換が発生しています。

`LLM2024_day4_raw.txt`の中には、このような誤変換が多数見られます。これらの誤りはRAG性能に直接影響し、モデルの回答精度を低下させる要因となります。

したがって、**ドキュメント品質の改善**を行い、RAG性能の向上を図ります。

## 講義内容をソースとして活用：改善版RAG実装

* **ドキュメント処理**:
  - speech2textによる書き起こしテキストを人手で丁寧に修正
  - 専門用語（Inference Time Scaling、GPT-o1など）の正確な表記を確保
  - 文脈の流れを維持しつつ、文法的に正確な日本語に修正

* **検索手法**:
  - 引き続き「。」（句点）で区切られた文単位でテキストを分割
  - 文単位の検索により、モデルの入力トークン制限内で関連情報を最大化

この改善により、モデルが正確な情報に基づいて「Inference Time Scaling」の概念を理解し、適切な回答を生成することが期待されます。

In [61]:
with open("/content/lecture-ai-engineering/day3/data/LLM2024_day4.txt", "r") as f:
  raw_writedown = f.read()

In [62]:
# ドキュメントを用意する。
documents = [text.strip() for text in raw_writedown.split("。")]
print("ドキュメントサイズ: ", len(documents))
print("ドキュメントの例: \n", documents[310])

ドキュメントサイズ:  350
ドキュメントの例: 
 それからBest of Nとはちょっと違う方法として、N個を生成した後に、それらを集約するという意味では、Day2でやったSelf-Consistencyをこの枠組みの一つとして説明されます


In [63]:
# Retrievalの実行
question = "LLMにおけるInference Time Scalingとは？"

query_embeddings = emb_model.encode([question], prompt_name="query")
document_embeddings = emb_model.encode(documents)

# 各ドキュメントの類似度スコア
scores = (query_embeddings @ document_embeddings.T) * 100
print(scores.tolist())

[[61.82579803466797, 66.46407318115234, 60.238338470458984, 51.8336067199707, 54.910701751708984, 51.560462951660156, 58.39971923828125, 60.80598068237305, 55.071990966796875, 59.28535461425781, 58.47712326049805, 62.86029815673828, 58.223690032958984, 59.106510162353516, 55.34396743774414, 57.35869598388672, 59.76055145263672, 51.49692153930664, 60.62431335449219, 62.524192810058594, 57.50200271606445, 58.56460189819336, 59.72063446044922, 59.440303802490234, 63.48786163330078, 61.78029251098633, 59.621185302734375, 62.459354400634766, 53.737796783447266, 61.90656280517578, 55.26152420043945, 56.99125671386719, 62.45269775390625, 56.23672103881836, 56.530357360839844, 56.4011344909668, 58.69480514526367, 51.75259017944336, 56.227516174316406, 56.70081329345703, 58.36994552612305, 55.44430160522461, 59.27916717529297, 59.5364990234375, 57.694984436035156, 54.76824188232422, 58.027793884277344, 57.40224838256836, 56.45027160644531, 52.43673324584961, 61.598026275634766, 62.7029876708984

In [64]:
topk = 5
for i, index in enumerate(scores.argsort()[0][::-1][:topk]):
  print(f"取得したドキュメント{i+1}: (Score: {scores[0][index]})")
  print(documents[index], "\n\n")

取得したドキュメント1: (Score: 67.38101196289062)
最後に補足して僕のパート終わろうと思いますけど、同じ計算資源のときにパラメータ増やすのよりも推論資源を増やすのが有効なのかっていうのが問いとしてあると思いますけど、o1の場合だと、訓練時のスケールは同じままって推論時のスケールを増やしたら、より賢くなりましたって話でしたけど、どっちにするのがいいのかっていう意味で言うと、GoogleDeepMindが8月に論文としてまして、Scaling LLM Test-Time Comupte Optimally can be more Effective than Scaling More Paremetersっていうことで、良いらしいというふうに言われてます 


取得したドキュメント2: (Score: 66.58785247802734)
右側がtest-time computeっていうふうに書いてると思うんすけど、推論時に計算資源を増やしたときあるモデルを使うんだけど、簡単に答える方法と深く考えて答える方法みたいでだんだん計算資源を増やしていったときに、性能がどう変わるかっていうので、これもスケールしていってるということがわかると思います 


取得したドキュメント3: (Score: 66.46407318115234)
あのスケールするっていうところではタイトルの通りなんですけど、ちょっとこれスケーリングPretraining回ってなってるんですけれども、ちょっと最近はですね、このPretrainingだけではなくて、推論をスケールさせるというような話も出てきてましてせっかくなのでその最近の話題ということです推論時のスケーリングことで、ちょっとタイトル詐欺が入ってるんですけどPretrainingだけじゃない、スケーリングも扱うということで、ちょっと若干あのタイトル詐欺なんですけども、あの最近の話題ということで推論時のスケジュールについても話していきたいなと思っています 


取得したドキュメント4: (Score: 66.20873260498047)
Trasnformerの場合はスケール則が、パラメータ数が横軸になってますけどこういうふうになると、LSTMの場合には1層2層4層みたいにそれぞれスケール則を解くとこんなふうになります

In [65]:
 #回答に役立つ該当の発言はreference[1871]〜に含まれてます。
references = "\n".join(["* " + documents[i] for i in scores.argsort()[0][::-1][:topk]])
messages = [
    {"role": "system", "content": "質問に回答してください。必ず「日本語で回答」すること。また、与えられる資料を参考にして回答すること。"},
    {"role": "user", "content": f"[参考資料]\n{references}\n\n[質問] LLMにおけるInference Time Scalingとは？"},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    # max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=False,
    # temperature=0.6, # If do_sample=True
    # top_p=0.9,  # If do_sample=True
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


In [66]:
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

LLM（Large Language Model）におけるInference Time Scalingは、推論時（Inference）に計算資源をスケーリングすることで、モデルがより賢くなり、性能が向上することを意味します。Google DeepMindの論文「Scaling LLM Test-Time Compute Optimally can be more Effective than Scaling More Parameters」では、推論時計算資源をスケーリングすることが、パラメータ数を増やすよりも有効な方法であると示されています。

Inference Time Scalingは、TrasnformerやLSTMなどのモデルにおいて、パラメータ数を増やすのではなく、推論時計算資源をスケーリングすることで、性能が向上することを目指します。例えば、Llama3の論文では、8億、70億、405億という異なるスケールで、モデルが性能を向上させることが示されています。

Inference Time Scalingは、最近の話題であり、Pretrainingだけでなく、推論時スケーリングも重要なトピックとなっています。


## 結果 (修正テキストによるRAG)

書き起こしテキストの品質改善により、モデルの回答に部分的な向上が見られました：

### 改善点
* 「推論時（Inference）に計算資源をスケーリングすることで、モデルがより賢くなり、性能が向上すること」という概念を正確に捉えるようになった

### 問題点
* 「Inference Time Scalingは、TransformerやLSTMなどのモデルにおいて、パラメータ数を増やすのではなく、推論時計算資源をスケーリングすることで、性能が向上すること...」という記述は講義内容と矛盾している

### 問題分析

モデルが誤った回答を生成する主要因として、**文脈の欠如**が考えられます：
* 「。」で区切られた短い文単位での検索では、各文の発言背景や関連性が失われる
* 単独の文から情報を抽出するため、講師の全体的な主張や議論の流れを把握できない
* 結果として、正しい個別の文でも、その解釈に必要な背景情報が欠如し、誤った文脈で理解される


---

# 3. 文脈を考慮したチャンク化の導入

検索結果の品質向上のため、以下の改善を実施します：

* **前後文脈を含むチャンク化**:
  - 検索でマッチした文だけでなく、その前後の複数文も含めてチャンクとして取得
  - 具体的には、マッチした文を中心に前2文、後2文を含む計5文程度のチャンクを構成
  - この「文脈ウィンドウ」により、発言の背景情報や議論の流れが保持される

* **期待される効果**:
  - 講師の主張とその根拠の関係性を正確に把握できる
  - 概念の定義とその適用範囲を正しく理解できる

この改善により、モデルが講義内容の本質をより正確に理解し、一貫性のある事実に基づいた回答を生成することが期待されます。

In [67]:
# 前後それぞれ2つずつの文章を一つのドキュメントに追加する。（要は5つの文章集合になる)
references = "\n".join(["* " + "。".join(documents[max(0, i-2): min(i+2, len(documents))]).strip() for i in scores.argsort()[0][::-1][:topk]])
messages = [
    {"role": "system", "content": "質問に回答してください。必ず「日本語で回答」すること。また、与えられる資料を参考にして回答すること。"},
    {"role": "user", "content": f"[参考資料]\n{references}\n\n[質問] LLMにおけるInference Time Scalingとは？"},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    # max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=False,
    # temperature=0.6, # If do_sample=True
    # top_p=0.9,  # If do_sample=True
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


In [ ]:
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

## 結果 (文脈付きチャンク化によるRAG)

文脈を含むチャンク化により、モデルの回答の方向性に明確な改善が見られました：

### 改善点
* 「推論時の計算をスケールさせる」という概念を据えて回答
* Inference Time Scalingの基本原理についての理解が向上

### 残存する問題点
* 質問と関連性の低い情報（ノイズ）が混入する

### 問題分析

文脈付きチャンク化によるアプローチで新たに発生した課題：

1. **情報過多の問題**:
   * ドキュメント量の増加により、モデルに提供される情報総量が大幅に増加
   * 関連情報と非関連情報が混在し、ノイズと重要情報の区別が困難に

2. **情報選択の複雑化**:
   * モデルは単に回答を生成するだけでなく、提供された多様な情報源から関連性の高い情報を選別する作業も担うことになった
   * この二重タスクにより回答生成の難易度が上昇


---

# 4. Rerankによる情報品質の向上

検索精度をさらに向上させるため、二段階の検索プロセスを導入します：

* **Rerank手法の導入**:
  - 第一段階: 従来通り基本的な検索アルゴリズムでtop-k個のドキュメントチャンクを取得
  - 第二段階: 取得したチャンクに対してLLMを活用した高度な関連性評価を実施
  - LLMに「このドキュメントは質問『LLMにおけるInference Time Scalingとは？』に対して本当に関連性が高いか」を判断させる
  - 関連性スコアに基づいてランク付けし、真に関連性の高いチャンクのみを選出

* **期待される効果**:
  - 質の高い情報に焦点を絞ることで、ノイズとなる情報を大幅に削減
  - 文脈を保ちながらも、関連性の高い情報のみをモデルに提供
  - モデルのタスクを「多量の情報から選別して回答」から「厳選された情報に基づいて回答」へと単純化

この高度な情報フィルタリングにより、Inference Time Scalingに関する正確で一貫性のある回答生成が期待されます。

In [ ]:
 #回答に役立つ該当の発言はreference[1871]〜に含まれてます。
references = []
for ref in ["。".join(documents[max(0, i-2): min(i+2, len(documents))]).strip() for i in scores.argsort()[0][::-1][:topk]]:
  messages = [
      {"role": "system", "content": "与えられた参考資料が質問に直接関連しているか？'yes''no'で答えること。ただし、余計なテキストを生成しないこと。"},
      {"role": "user", "content": f"[参考資料]\n{ref}\n\n[質問] LLMにおけるInference Time Scalingとは？"},
  ]
  input_ids = tokenizer.apply_chat_template(
      messages,
      add_generation_prompt=True,
      return_tensors="pt"
  ).to(model.device)

  terminators = [
      tokenizer.eos_token_id,
      tokenizer.convert_tokens_to_ids("<|eot_id|>")
  ]

  outputs = model.generate(
      input_ids,
      # max_new_tokens=256,
      eos_token_id=terminators,
      do_sample=False,
      # temperature=0.6, # If do_sample=True
      # top_p=0.9,  # If do_sample=True
  )

  response = outputs[0][input_ids.shape[-1]:]
  response = tokenizer.decode(response, skip_special_tokens=True)
  print("\n\n対象となるドキュメント:\n", ref.replace("。", "。\n"))
  print("\n関連しているかどうか: ", response)

  if "yes" in response.lower():
    references.append(ref)

In [ ]:
print(len(references))

上記より、上位3件のみが関連しているとわかったので、これらだけをモデルに渡すこととする。

In [ ]:
 #回答に役立つ該当の発言はreference[1871]〜に含まれてます。
messages = [
    {"role": "system", "content": "質問に回答してください。必ず「日本語で回答」すること。また、与えられる資料を参考にして回答すること。"},
    {"role": "user", "content": f"[参考資料]\n{references}\n\n[質問] LLMにおけるInference Time Scalingとは？"},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    # max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=False,
    # temperature=0.6, # If do_sample=True
    # top_p=0.9,  # If do_sample=True
)

In [ ]:
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

## 結果 (Rerank導入後)

Rerankの導入により、回答品質に改善が見られました：

### 達成された成果
* Inference Time Scalingに関する正確な情報を含んだ回答の生成
* 無関係な情報やノイズの排除
* 講義内容を反映した説明の実現 🎉

この結果から、RAGパイプラインにおける情報の質と関連性の重要性であり、検索で取得した情報を単に増やすだけでなく、その情報の関連性を精査する方法を学ぶことができました。

---


# 5. さらなる改善案: 意味的チャンク化

文単位での分割と前後文脈の追加という現在のアプローチをさらに発展させる手法として、**意味的なチャンク化**が考えられます：

* **意味的チャンク（段落）単位での分割**:
  - 単純な文の区切りではなく、意味的なまとまり（トピック、議論、例示など）に基づいてテキストを分割
  - 人間の主観に基づく意味的な段落分けを活用
  - 各チャンクが「一つの完結した考え」を表現するようにする

* **期待される効果**:
  - より自然な文脈理解が可能に（人間の思考や会話の流れに近い）
  - トピックの開始から結論までの流れを維持できる
  - 概念間の関係性や比較が同一チャンク内に含まれ、より深い理解につなげる

* **検証方法**:
  - 人間が主観的に意味でグループ化したチャンクセットを用意
  - 同じRerank手法を適用し、文単位チャンクとの性能差を比較
  - 回答の正確性、一貫性、網羅性を評価指標として使用

この意味的チャンク化手法は、特に講義のような構造化された発話においては、より自然で効果的な情報検索と理解を可能にすると予想されます。

In [ ]:
# 本来は段落をそのままdocumentsに入れずに一定のサイズに分割した方が良いでしょうが、簡単のために段落をそのまま入れてしまいます。
documents = [text.replace("\n", " ").strip() for text in raw_writedown.split("\n\n")]
print("ドキュメントサイズ: ", len(documents))
print("ドキュメントの例: \n", documents[30])

In [ ]:
question = "LLMにおけるInference Time Scalingとは？"

query_embeddings = emb_model.encode([question], prompt_name="query")
document_embeddings = emb_model.encode(documents)

scores = (query_embeddings @ document_embeddings.T) * 100
print(scores.tolist())

In [ ]:
# 簡単のためにtop2でやります。結果を見てもらえれば問題なく関連する項目のみ取得できているのが分かるかと思います。
topk = 2
for i, index in enumerate(scores.argsort()[0][::-1][:topk]):
  print(f"取得したドキュメント{i+1}: (Score: {scores[0][index]})")
  print(documents[index], "\n\n")

In [ ]:
reference = "\n".join(["* " + documents[i] for i in scores.argsort()[0][::-1][:topk]])

messages = [
    {"role": "system", "content": "質問に回答してください。必ず「日本語で回答」すること。また、与えられる資料を参考にして回答すること。"},
    {"role": "user", "content": f"[参考資料]\n{reference}\n\n[質問] LLMにおけるInference Time Scalingとは？"},
]
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    # max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=False,
    # temperature=0.6, # If do_sample=True
    # top_p=0.9,  # If do_sample=True
)

In [ ]:
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))